OpenAI’s API gives practitioners access to GPT-3, an incredibly powerful natural language model that can be applied to virtually any task that involves understanding or generating natural language.

If you use OpenAI's API to [fine-tune GPT-3](https://beta.openai.com/docs/guides/fine-tuning), you can now use the W&B integration to track experiments, models, and datasets in your central dashboard.

All it takes is one line: `openai wandb sync`

## Set up your API key

In [19]:
# Enter credentials
%env = "NA"

env: ="NA"


**WARNING: Remove the API key after running the cell and clear output so it does not get logged to wandb in case you sync code (see settings)**

## Install dependencies

In [ ]:
!pip install -Uq openai wandb

     |████████████████████████████████| 44 kB 1.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.9 MB 9.7 MB/s 
     |████████████████████████████████| 146 kB 49.8 MB/s 
     |████████████████████████████████| 182 kB 43.2 MB/s 
     |████████████████████████████████| 168 kB 32.7 MB/s 
     |████████████████████████████████| 62 kB 583 kB/s 
     |████████████████████████████████| 168 kB 48.5 MB/s 
     |████████████████████████████████| 166 kB 42.8 MB/s 
     |████████████████████████████████| 166 kB 52.8 MB/s 
     |████████████████████████████████| 162 kB 34.0 MB/s 
     |████████████████████████████████| 162 kB 33.9 MB/s 
     |████████████████████████████████| 158 kB 75.2 MB/s 
     |████████████████████████████████| 157 kB 39.5 MB/s 
     |████████████████████████████████| 157 kB 40.1 MB/s 
     |████████████████████████████████| 157 kB 46.4 MB/s 
     |███

In [ ]:
import openai
import wandb
from pathlib import Path
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [ ]:
run = wandb.init(project='GPT 3 for Generating Breaking Bad Dialog', job_type="dataset_preparation")

Using Weights & Biases Artifacts to Download a .CSV dataset file with episode title > synopsis pairs

In [ ]:
run = wandb.init(project='GPT 3 for Generating Breaking Bad Dialog')
artifact = run.use_artifact('ivangoncharov/GPT-3 to Generate Doctor Who Synopses/dw_synopses_csv:v0', type='raw_dataset')
artifact.download()

wandb:   1 of 1 files downloaded.  


In [ ]:
artifact_dir ="./artifacts/BreakingCSV.csv"

In [ ]:
#Shuffling the dataset with fixed seed

df = pd.read_csv(artifact_dir)
ds = df.sample(frac=1, random_state=0)


wandb.init(project="GPT 3 for Generating Breaking Bad Dialog", job_type="logging_dataset_as_table")
wandb.run.log({"Raw dataset" : wandb.Table(dataframe=ds)})

ds.to_csv("BreakingCSV.csv")
ds.head()

,prompt,completion
6638,"I mean, that's the game here. ->",You gotta have your family packed and ready to...
7975,What? Please tell me that's not what I think i...,Has anything like this ever happened before? N...
5915,I'm very anxious to find out what this is all ...,"Sir, your fingerprints were found at a crime s..."
8050,"And I'm telling you these days, chances of get...",It's like Bigfoot. END
5894,My wife and I are thinking about having kids. ->,"Walk into a doctor's office, and suddenly, boo..."


### Using OpenAI Tool to preprocess the data

In [ ]:
!openai tools fine_tunes.prepare_data -f BreakingCSV.csv

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 10886 prompt-completion pairs
- The input file should contain exactly two columns/keys per row. Additional columns/keys present are: ['Unnamed: 0']
- There are 59 duplicated prompt-completion sets. These are rows: [984, 1321, 1692, 2101, 2213, 2235, 2565, 2664, 3080, 3282, 3969, 4184, 4373, 4659, 5049, 5128, 5278, 5326, 5488, 5619, 6025, 6047, 6102, 6466, 6535, 6619, 6660, 6741, 6766, 7315, 7406, 7428, 7624, 8061, 8164, 8359, 8369, 8465, 8600, 8613, 8646, 8778, 8793, 9084, 9436, 9816, 9887, 9927, 9934, 10429, 10450, 10536, 10606, 10633, 10700, 10703, 10730, 10811, 10868]
- All prompts end with suffix ` ->`
- All completions end with suffix ` END`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the a

### Splitting the data into train and val sets

In [ ]:
#The dataset has 304 pairs in total
import json

with open('BreakingCSV_prepared.jsonl', 'r') as json_file:
    json_list = list(json_file)

num_data = len(json_list)
print("Total:", num_data)

val_part = 0.1 

val_amount = int(num_data * val_part)
print("Val data:", val_amount)
train_amount = num_data - val_amount 
print("Train data:", train_amount)

!head -n $train_amount BreakingCSV_prepared.jsonl > bb_train.jsonl
!tail -n $val_amount  BreakingCSV_prepared.jsonl > bb_valid.jsonl


Total: 10827
Val data: 1082
Train data: 9745


In [ ]:
wandb.finish()

### Let's define our GPT-3 fine-tuning hyper-parameters.

In [ ]:
model = 'curie'  # can be ada, babbage or curie
n_epochs = 4
batch_size = 4
learning_rate_multiplier = 0.1
prompt_loss_weight = 0.1

### Time to train the model!

In [ ]:
!openai api fine_tunes.create \
    -t bb_train.jsonl \
    -v bb_valid.jsonl \
    -m $model \
    --n_epochs $n_epochs \
    --batch_size $batch_size \
    --learning_rate_multiplier $learning_rate_multiplier \
    --prompt_loss_weight $prompt_loss_weight

Upload progress: 100% 1.34M/1.34M [00:00<00:00, 1.42Git/s]
Uploaded file from bb_train.jsonl: file-4gbrL7hmLhB6exp6KxwvtkMh
Upload progress: 100% 146k/146k [00:00<00:00, 178Mit/s]
Uploaded file from bb_valid.jsonl: file-oxT5tAHZDXxrymOngMtUi7g4
Created fine-tune: ft-ihyVLAkmTeMkMfFJ24nxxNXM
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-11-17 06:08:39] Created fine-tune: ft-ihyVLAkmTeMkMfFJ24nxxNXM
[2022-11-17 06:08:45] Fine-tune costs $3.35
[2022-11-17 06:08:45] Fine-tune enqueued. Queue number: 0
[2022-11-17 06:08:46] Fine-tune started

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-ihyVLAkmTeMkMfFJ24nxxNXM



In [ ]:
!openai api fine_tunes.follow -i ft-ihyVLAkmTeMkMfFJ24nxxNXM

[2022-11-17 06:08:39] Created fine-tune: ft-ihyVLAkmTeMkMfFJ24nxxNXM
[2022-11-17 06:08:45] Fine-tune costs $3.35
[2022-11-17 06:08:45] Fine-tune enqueued. Queue number: 0
[2022-11-17 06:08:46] Fine-tune started
[2022-11-17 06:32:18] Completed epoch 1/4
[2022-11-17 06:55:04] Completed epoch 2/4
[2022-11-17 07:17:41] Completed epoch 3/4
[2022-11-17 07:40:20] Completed epoch 4/4
[2022-11-17 07:40:45] Uploaded model: curie:ft-personal-2022-11-17-07-40-44
[2022-11-17 07:40:46] Uploaded result file: file-kG5P8EC6k3POdA0zEIBA7wVp
[2022-11-17 07:40:46] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2022-11-17-07-40-44 -p <YOUR_PROMPT>


In [ ]:
!openai api completions.create -m curie:ft-personal-2022-11-17-07-40-44 -p "How do you know my name? ->"

How do you know my name? -> Your Aunt Marie knows. She's been giving me a ton of your history.

In [ ]:
!openai api completions.create -m curie:ft-personal-2022-11-17-01-49-03 -p "How do you know my name?"

How do you know my name? - Your name was on the list.
 Whoa, whoa,

## Sync fine-tune jobs to Weights & Biases

We can log our fine-tunes with a simple command.

In [ ]:
!openai wandb sync --help

usage: openai wandb sync [-h] [-i ID] [-n N_FINE_TUNES] [--project PROJECT]
                         [--entity ENTITY] [--force]

optional arguments:
  -h, --help            show this help message and exit
  -i ID, --id ID        The id of the fine-tune job (optional)
  -n N_FINE_TUNES, --n_fine_tunes N_FINE_TUNES
                        Number of most recent fine-tunes to log when an id is
                        not provided. By default, every fine-tune is synced.
  --project PROJECT     Name of the project where you're sending runs. By
                        default, it is "GPT-3".
  --entity ENTITY       Username or team name where you're sending runs. By
                        default, your default entity is used, which is usually
                        your username.
  --force               Forces logging and overwrite existing wandb run of the
                        same fine-tune.


In [ ]:
!openai wandb sync --project "GPT 3 for Generating Breaking Bad Dialog" 

wandb: Currently logged in as: jareyna. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in /content/wandb/run-20221117_074332-ft-ihyVLAkmTeMkMfFJ24nxxNXM
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-ihyVLAkmTeMkMfFJ24nxxNXM
wandb: ⭐️ View project at https://wandb.ai/jareyna/GPT%203%20for%20Generating%20Breaking%20Bad%20Dialog
wandb: 🚀 View run at https://wandb.ai/jareyna/GPT%203%20for%20Generating%20Breaking%20Bad%20Dialog/runs/ft-ihyVLAkmTeMkMfFJ24nxxNXM
File file-4gbrL7hmLhB6exp6KxwvtkMh could not be retrieved. Make sure you are allowed to download training/validation files
File file-oxT5tAHZDXxrymOngMtUi7g4 could not be retrieved. Make sure you are allowed to download training/validation files
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_toke

Anytime we have new fine-tunes, we can just call openai wandb sync to add them to our dashboard.

## Log inference samples

The best way to evaluate a generative model is to explore sample predictions.

Let's generate a few inference samples and log them to W&B.

In [ ]:
# create eval job
run = wandb.init(project='GPT 3 for Generating Breaking Bad Dialog', job_type='eval')
entity = wandb.run.entity

wandb: Currently logged in as: jareyna. Use `wandb login --relogin` to force relogin


We can easily retrieve all config parameters from a job file.

Job files are logged to W&B as artifacts and can be accessed with `run.use_artifact('USERNAME/PROJECT/job_details:VERSION')` where `VERSION` is either:
* a version number such as `v2`
* the fine-tune id such as `ft-xxxxxxxxx`
* an alias added automatically such as `latest` or manually

You can explore them in your artifacts dashboard.

In [ ]:
# choose a fine-tuned model
artifact_job = run.use_artifact(f'{entity}/GPT 3 for Generating Breaking Bad Dialog/fine_tune_details:latest', type='fine_tune_details')
artifact_job.metadata



{'id': 'ft-ihyVLAkmTeMkMfFJ24nxxNXM',
 'model': 'curie',
 'object': 'fine-tune',
 'status': 'succeeded',
 'created_at': 1668665319,
 'updated_at': 1668670846,
 'hyperparams': {'n_epochs': 4,
  'batch_size': 4,
  'prompt_loss_weight': 0.1,
  'learning_rate_multiplier': 0.1},
 'result_files': [{'id': 'file-kG5P8EC6k3POdA0zEIBA7wVp',
   'bytes': 641022,
   'object': 'file',
   'status': 'processed',
   'purpose': 'fine-tune-results',
   'filename': 'compiled_results.csv',
   'created_at': 1668670845,
   'status_details': None}],
 'training_files': [{'id': 'file-4gbrL7hmLhB6exp6KxwvtkMh',
   'bytes': 1342420,
   'object': 'file',
   'status': 'processed',
   'purpose': 'fine-tune',
   'filename': 'bb_train.jsonl',
   'created_at': 1668665318,
   'status_details': None}],
 'organization_id': 'org-m9no01INvISPT0msAk4Cycqv',
 'fine_tuned_model': 'curie:ft-personal-2022-11-17-07-40-44',
 'validation_files': [{'id': 'file-oxT5tAHZDXxrymOngMtUi7g4',
   'bytes': 145924,
   'object': 'file',
   's

Let's take advantage to add metadata into our eval run config.

In [ ]:
wandb.config.update({k:artifact_job.metadata[k] for k in ['fine_tuned_model', 'model', 'hyperparams']})

We can easily access model id from any job.

In [ ]:
fine_tuned_model = artifact_job.metadata['fine_tuned_model']
fine_tuned_model

'curie:ft-personal-2022-11-17-07-40-44'

Loading validation data as dataframe

In [ ]:
df = pd.read_json("bb_valid.jsonl", orient='records', lines=True)
df.head()

,prompt,completion
0,"But what? The rest of us are like, open season...",Enough. END
1,Yes. ->,The owner is quite the collector. END
2,It's about 700 grand. That's a pretty good hau...,"As I say, it's a good day for the citizens of..."
3,"And, no doubt, our profits will be smaller. ->",At least at first. END
4,Ohh Oi. ->,Wait a minute. END


We'll perform the inference on all 30 validation examples. 

In [ ]:
n_samples = 30
df = df.iloc[:n_samples]

In [ ]:
data = []

for _, row in tqdm(df.iterrows()):
    prompt = row['prompt']
    res = openai.Completion.create(model=fine_tuned_model, prompt=prompt, max_tokens=300, stop=[" END"])
    completion = res['choices'][0]['text']
    completion = completion[1:]       # remove initial space
    prompt = prompt[:-3]              # remove " ->"
    target = row['completion'][1:-4]  # remove initial space and "END"
    data.append([prompt, target, completion])

30it [00:07,  3.80it/s]


We create and log a W&B Table to easily explore, query & compare model predictions.

In [ ]:
prediction_table = wandb.Table(columns=['prompt', 'target', 'completion'], data=data)

In [ ]:
wandb.log({'predictions': prediction_table})

You can open the link to your run page down below. 

In [ ]:
wandb.finish() #work out a way to print the run page link

# Resources
https://wandb.ai/ivangoncharov/GPT-3%20to%20Generate%20Doctor%20Who%20Synopses?workspace=user-ivangoncharov